In [1]:
import os
from arcgis.gis import GIS
from arcgis.env import active_gis
from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
from arcgis import geoenrichment
import pandas as pd
import numpy as np

import sys
sys.path.append('../src/geoai_retail')
import utils
import config
from enrich_rest import enrich, get_online_variable_list

ent_gis = GIS(config.ent_url, username=config.ent_user, password=config.ent_pass)
ent_gis

GIS @ https://geoai-webgis-ent.bd.esri.com/portal version:7.1

In [11]:
gis = ent_gis
gis._con.relogin()

'VoeAmzcqtpo2Op7iHUQWbvxSUsC0nFXEsFsEaZO5x2VZ9K-iu99sBp7WdFn87CsN4XdsN_PkzUMgokFh41gtWHYuXYs6dujvtpb7lsy9Hg15a2G7WQK2Rgs_5BpZRHWA'

In [10]:
gis._con.token

'PippaRZJ1MbJOz4qi__HKO-OKJyxhYPO_x1j2SsrsNILi_-r4IRqrSaWGXzZKgAvaVGbu0Z8S8F2T7KFSFm-cnpnq2AU7jK2LmX0LLOXPjJ4S8oHG8pnWPlYT4GigB-a'

In [12]:
gis.properties.helperServices.geoenrichment.url

'https://geoai-webgis-ba.bd.esri.com/server/rest/services/World/GeoEnrichmentServer'

In [2]:
raw_gdb = r'../data/raw/raw.gdb'
origin_fc = os.path.join(raw_gdb, 'blocks')
origin_id_fld = 'GEOID'

geo_fc = origin_fc
geo_id_fld = origin_id_fld

vars_csv = '../data/raw/enrichment_variables.csv'

In [ ]:
from collections import namedtuple
from functools import reduce
import tempfile
import uuid

# location to store temp files if necessary
csv_file_prefix = 'temp_enrich'
temp_file_root = os.path.join(tempfile.gettempdir(), csv_file_prefix)


def get_online_variable_list(variable_list_to_lookup, gis, exclude_five_year_projections=True):
    """
    From an input variable list derived from a dataset already enriched, use ArcGIS Online to look up a list of enrichment
        variables to use for performing enrichment on another new dataset.
    :param variable_list_to_lookup: List of field/column names from an dataset already enriched.
    :param exclude_five_year_projections: Boolean to indicate if to consider the future year (five year projection) variables.
    """
    # get the data available in the USA
    usa = geoenrichment.Country.get('USA')
    enrich_df = usa.data_collections
    
    # if excluding the five year projections, drop them out
    if exclude_five_year_projections:
        five_year_out = str(np.max([int(val) for val in enrich_df['vintage'].unique() if pd.notna(val) and not '-' in val]))
        enrich_df = enrich_df[enrich_df['vintage'] != five_year_out].copy()
    
    # slice off the variable name in a separate column
    enrich_df['variable_name'] = enrich_df['analysisVariable'].apply(lambda val: val.split('.')[1])
    
    # drop any variable name duplicates
    enrich_df = enrich_df.drop_duplicates('variable_name')
    
    # flag variables we want to use for enriching the new data
    var_match = enrich_df['variable_name'].apply(lambda var_name: var_name in variable_list_to_lookup)
    
    # create a list of the analysis variables usable for enrichment
    return list(enrich_df[var_match]['analysisVariable'].values)


def get_enrichment_alias_dataframe(variable_list_to_lookup, gis):
    
    # get the data available in the USA
    usa = geoenrichment.Country.get('USA')
    enrich_df = usa.data_collections
    
    # slice off the variable name in a separate column
    enrich_df['variable_name'] = enrich_df['analysisVariable'].apply(lambda val: val.split('.')[1])
    
    # drop any variable name duplicates
    enrich_df = enrich_df.drop_duplicates('variable_name')
    
    # flag variables from the input
    var_match = enrich_df['variable_name'].apply(lambda var_name: var_name in variable_list_to_lookup)
    
    # create a dataframe of just the matching variables
    alias_df = enrich_df[var_match]
    
    # filter out only needed columns
    alias_df = alias_df[['variable_name', 'analysisVariable', 'alias']]
    
    # rename columns
    alias_df.columns = ['variable_name', 'enrichment_name', 'alias_name']
    
    return alias_df.reset_index(drop=True)


def get_enrich_limits(gis):

    # get the limitations for the enrichment service
    limits_resp = gis._con.get(f'{gis.properties.helperServices.geoenrichment.url}/Geoenrichment/ServiceLimits')

    # extract out and reformat the limits
    limits_lst = limits_resp['serviceLimits']['value']
    limits_dict = {itm['paramName']: itm['value'] for itm in limits_lst}

    # save the values into a named tuple
    EnrichLimits = namedtuple('EnrichLimits', ['max_record_count', 'max_collections', 'max_fields'])
    enrich_limits = EnrichLimits(limits_dict['maxRecordCount'], limits_dict['maximumDataCollections'], 
                                 limits_dict['maximumOutFieldsNumber'])
    return enrich_limits


def _enrich_to_csv(geo_df, var_lst, gis):
    temp_csv = f'{temp_file_root}_{uuid.uuid4().hex}.csv'
    enrich_df = _enrich_wrapper(geo_df, var_lst, gis)
    enrich_df.to_csv(temp_csv)
    return temp_csv

def _enrich_wrapper(geo_df, variable_lst, gis):
    enrich_limits = get_enrich_limits(gis)

    var_df = pd.DataFrame([[var] + var.split('.') for var in enrich_lst], columns=['enrich_var', 'collection', 'var_name'])
    collections = var_df.collection.unique()
    collection_cnt = collections.size

    if collection_cnt > enrich_limits.max_collections:

        enrich_var_blocks = [list(var_df[var_df['collection'].isin(collections[idx:idx + enrich_limits.max_collections])]['enrich_var'].values)
                             for idx in range(0, collection_cnt,  enrich_limits.max_collections)]

        enrich_df_lst = [geoenrichment.enrich(geo_df.copy(), analysis_variables=enrich_block, return_geometry=False)
                         for enrich_block in enrich_var_blocks]

        enrich_df = reduce(lambda left,right: pd.merge(left,right), enrich_df_lst)

        enrich_df = enrich_df[[geo_id_fld] + list(var_df['var_name'])].copy()
        
        return enrich_df

    
def enrich(input_data, input_data_id_col, variable_list, gis):
        # get the data into a dataframe
        geo_df = utils.get_dataframe(input_data)
        
        # get the limitations on the enrichment rest endpoint, and scale the analysis based on this
        enrich_limits = get_enrich_limits(gis)
        max_records = enrich_limits.max_record_count

        # if necessary, batch the analysis based on the size of the input data, and the number of destinations per origin
        if len(geo_df.index) > max_records:

            # process each batch, and save the results to a temp file in the temp directory
            enrich_csv_list = [_enrich_to_csv(geo_df.iloc[idx:idx + max_records], variable_list, gis)
                                for idx in range(0, len(origin_df.index), max_origin_cnt)]

            # load all the temporary files into dataframes and combine them into a single dataframe
            enrich_df = pd.concat([pd.read_csv(enrich_csv) for enrich_csv in enrich_csv_list])

            # clean up the temp files
            for csv_file in enrich_csv_list:
                os.remove(csv_file)

        else:
            enrich_df = _enrich_wrapper(geo_df, variable_list, gis)
            
        # get a list of just the columns requested
        col_lst = [input_data_id_col] + [var.split('.')[1] for var in variable_list]
            
        return enrich_df[col_lst].copy()

In [3]:
geo_df = utils.get_dataframe(geo_fc)
geo_df = geo_df[[geo_id_fld, 'SHAPE']].copy()
geo_df.spatial.set_geometry('SHAPE')

,GEOID,SHAPE
0,410050201001000,"{""rings"": [[[-122.65566145899999, 45.426378462..."
1,410050201001001,"{""rings"": [[[-122.65534145899994, 45.427037462..."
2,410050201001002,"{""rings"": [[[-122.65548645999996, 45.429533462..."
3,410050201001003,"{""rings"": [[[-122.65696245999999, 45.428736462..."
4,410050201001004,"{""rings"": [[[-122.65702845999999, 45.428751462..."


In [4]:
vars_df = pd.read_csv(vars_csv, index_col=0)
vars_df['variable_name'] = vars_df['variable_name'].str.upper()
vars_df.drop_duplicates('variable_name')
var_lst = list(vars_df['variable_name'].values)
enrich_lst = get_online_variable_list(var_lst, ent_gis)

variable count = 1,627

['GQPOP_CY', 'GENALPHACY', 'GENZ_CY', 'MILLENN_CY', 'GENX_CY', 'BABYBOOMCY', 'OLDRGENSCY', 'GENBASE_CY', 'POP0_CY', 'POP5_CY', 'POP10_CY', 'POP15_CY', 'POP20_CY', 'POP25_CY', 'POP30_CY', 'POP35_CY', 'POP40_CY', 'POP45_CY', 'POP50_CY', 'POP55_CY']


In [ ]:
%%time
geo_df = geo_df_full.iloc[:1500].copy()
geo_df.spatial.set_geometry('SHAPE')
enrich_df = enrich(geo_df, geo_id_fld, enrich_lst, ent_gis)

In [ ]:
enrich_df

In [ ]:
%%time
enrich_df = enrich_data(geo_df, enrich_lst)

In [23]:





enrich_limits = get_enrich_limits(ent_gis)

var_df = pd.DataFrame([[var] + var.split('.') for var in enrich_lst], columns=['enrich_var', 'collection', 'var_name'])
collections = var_df.collection.unique()
collection_cnt = collections.size

if collection_cnt > enrich_limits.max_collections:
    
    enrich_var_blocks = [list(var_df[var_df['collection'].isin(collections[idx:idx + enrich_limits.max_collections])]['enrich_var'].values)
                         for idx in range(0, collection_cnt,  enrich_limits.max_collections)]
    
    enrich_df_lst = [enrich(geo_df.copy(), analysis_variables=enrich_block, return_geometry=False)
                     for enrich_block in enrich_var_blocks]
    
    enrich_df = reduce(lambda left,right: pd.merge(left,right), enrich_df_lst)
    
    enrich_df = enrich_df[[geo_id_fld] + list(var_df['var_name'])].copy()

enrich_df

,GEOID,FAGE0_CY,FAGE1_CY,FAGE2_CY,FAGE3_CY,FAGE4_CY,FAGE5_CY,FAGE6_CY,FAGE7_CY,FAGE8_CY,...,ACSUNT1ATT,ACSUNT2,ACSUNT3,ACSUNT5,ACSUNT10,ACSUNT20,ACSUNT50UP,ACSUNTMOB,ACSUNTOTH,ACSBLT2010
0,410050201001000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,410050201001001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,410050201001002,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
3,410050201001003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,410050201001004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,410050201001005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,410050201001006,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,410050201001007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,410050201001008,0,0,0,0,0,0,0,0,1,...,3,0,0,1,0,0,0,0,0,0
9,410050201001009,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
